## Emergency: Force Kill Stuck Ports

If your Jupyter kernel crashes and ports are still bound, use these terminal commands:

**macOS/Linux:**
```bash
# Find process using port 5001
lsof -ti:5001

# Kill process on port 5001
lsof -ti:5001 | xargs kill

# Force kill if needed
lsof -ti:5001 | xargs kill -9
```

**Or restart Jupyter kernel**: Kernel → Restart Kernel

## Auto-Reload in Python Scripts

For scripts (`.py` files), enable auto-reload using uvicorn directly:

**1. Structure your script with app at module level:**
```python
# my_bot.py
from pylogue.chatapp import create_default_chat_app

async def my_responder(msg: str, context=None) -> str:
    return f"Response: {msg}"

# Create app at module level (not inside if __name__)
app = create_default_chat_app(responder=my_responder)

if __name__ == "__main__":
    app.run(port=5001)
```

**2. Run with uvicorn for auto-reload:**
```bash
# From the directory containing my_bot.py
uvicorn my_bot:app --reload --port 5001

# Or with full path
uvicorn scripts.examples.basic-functionality.1-echo-bot:echo_app --reload --port 5001
```

Now any changes to the file will automatically restart the server!

# Running Pylogue in Jupyter Notebooks

This notebook shows how to run Pylogue chat applications directly in Jupyter notebooks using FastHTML's Jupyter utilities.

In [1]:
import asyncio
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from pylogue.chatapp import create_default_chat_app

## Method 1: Using JupyUvi (Recommended)

`JupyUvi` starts a uvicorn server in a background thread, perfect for Jupyter notebooks.

In [ ]:
# Define a simple responder
import random, time


async def my_responder(message: str, context=None) -> str:
    s = random.random() * 2
    await asyncio.sleep(s)
    return f"🤖 Response: {message.upper()}\n⏱️ Waited {s:.2f}s"


# Try to stop existing server
try:
    time.sleep(1)
    server.stop()
    # Wait for port to be free
    time.sleep(1)
except Exception as e:
    print(f"⚠️ Stop attempt: {e}")

# Create and start new server
app = create_default_chat_app(responder=my_responder)
port = 5005
server = JupyUvi(app.get_app(), port=port)
print(f"✅ Server running at http://localhost:{port}")

⚠️ Stop attempt: name 'server' is not defined


✅ Server running at http://localhost:5005


The server is now running! You can:
1. Access it at: http://localhost:5001
2. Display it in an iframe below (uncomment the next cell to see it)

**To make changes**: Just modify the `my_responder` function above and re-run the cell. The `restart_server()` helper will safely stop and restart.

In [3]:
# Display the app in an iframe within the notebook
# Uncomment the line below to see the app embedded in the notebook
# HTMX(port=5005)

## Stopping Servers

Always stop servers when you're done to free up ports:

In [4]:
# Stop all servers
server.stop()

TypeError: 'str' object cannot be interpreted as an integer

## Tips for Jupyter Development

### 1. Port Management
Each server needs a unique port. Use different ports (5001, 5002, etc.) for multiple apps.

### 2. Safe Server Restart Pattern
**Problem**: Calling `server.stop()` doesn't immediately free the port, causing "address already in use" errors.

**Solution**: Use a helper function with proper wait:
```python
import time

def restart_server(port=5001):
    global server
    try:
        if 'server' in globals() and server is not None:
            server.stop()
            time.sleep(1)  # Wait for port to free
    except: pass
    
    app = create_default_chat_app(responder=my_responder)
    server = JupyUvi(app.get_app(), port=port)
    return server

# Now you can safely rerun this cell
server = restart_server(port=5001)
```

### 3. Viewing Apps
- Use `HTMX()` to embed the app in an iframe within the notebook
- Or open http://localhost:PORT in your browser

### 4. Debugging
Server logs appear in the notebook output. Set `log_level="info"` for more details:
```python
server = JupyUvi(app.get_app(), port=5001, log_level="info")
```

### 5. Common Issues

**"Address already in use" error:**
- The port hasn't been freed yet after `stop()`
- Solution: Add `time.sleep(1)` after `server.stop()`
- Or use a different port temporarily

**Cell blocks during execution:**
- This happens when the server crashes but the port is still bound
- Solution: Restart the Jupyter kernel (Kernel → Restart)
- Or use `lsof -ti:5001 | xargs kill` in terminal to force-free the port

**Changes not reflected:**
- Make sure to stop and restart the server
- The server loads code once at startup
- Use the restart helper function pattern above

### 6. Development Workflow

**Best practice for rapid iteration:**
```python
# 1. Define responder in a cell
async def my_responder(msg: str, context=None) -> str:
    return f"Version 2: {msg}"

# 2. Restart in same cell or next cell
server = restart_server(port=5001)

# 3. Test in browser, then modify responder and re-run
```

**Alternative: Use different ports during development:**
```python
# Iteration 1
server1 = JupyUvi(app1.get_app(), port=5001)

# Iteration 2 (new port, no need to stop)
server2 = JupyUvi(app2.get_app(), port=5002)

# Clean up later
server1.stop()
server2.stop()
```

### 7. WebSocket Support
Pylogue's streaming works seamlessly in Jupyter notebooks!

## Next Steps

- Check `5-Examples.ipynb` for more responder patterns
- See `scripts/examples/ai/pydantic-ai.py` for LLM integration
- Read the documentation notebooks (0-Card through 4-ChatApp) for deep dives